In [1]:
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
import random
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemmatizer=WordNetLemmatizer()
import string
from nltk import pos_tag

In [3]:
from nltk.corpus import wordnet

In [4]:
import pandas as pd

In [5]:
import numpy as np

In [6]:
raw_data=pd.read_csv("PHONEPE_MASTERSHIVAM_SHANTANU.CSV",encoding="ISO-8859-1")

In [7]:
#raw_data.head()

In [8]:
#raw_data.tweet

In [9]:
from nltk.tokenize import word_tokenize

In [10]:
tweets_text=[]
for i,row in raw_data.iterrows():
    tweets_text.append(raw_data.at[i,'tweet'])

In [11]:
#tweets_text[1]

In [12]:
tokenized_doc=[]
for i in tweets_text:
    tokenized_doc.append(word_tokenize(i))

In [13]:
#tokenized_doc[1]

In [14]:
def get_simple_postag(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else :
        return wordnet.NOUN

In [15]:
stops=set(stopwords.words('english'))
punctuation=list(string.punctuation)
stops.update(punctuation)

In [16]:
def clean_reviews(words):
    output_words=[]
    for w in words:
        if w.lower() not in stops:
            pos=pos_tag([w])
            clean_word=(lemmatizer.lemmatize(w,get_simple_postag(pos[0][1])))
            output_words.append(clean_word.lower())
    return output_words

In [17]:
cleaned_tokenized_doc=[]
for i in tokenized_doc:
    cleaned_tokenized_doc.append(clean_reviews(i))

In [18]:
#cleaned_tokenized_doc[1]

In [19]:
#len(cleaned_tokenized_doc)

In [20]:
training_data=cleaned_tokenized_doc

In [21]:
#training_data[1]

In [22]:
#len(training_data)

In [23]:
training_data=[" ".join(i) for i in training_data]

In [24]:
#training_data[1]

In [25]:
count_vec=CountVectorizer(max_features=10000,ngram_range=(1,2),max_df=0.85,min_df=0.1)
a=count_vec.fit_transform(training_data)

In [26]:
data_dense=a.todense()

In [27]:
#print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

In [28]:
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD

In [31]:
#def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
 #   for topic_idx, topic in enumerate(H):
  #      print("Topic %d:" % (topic_idx))
   #     print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
    #    top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
     #   for doc_index in top_doc_indices:
      #      print (documents[doc_index])

In [33]:
#display_topics(H,W,feature_names,training_data,4,4)

In [34]:
from sklearn.model_selection import GridSearchCV

In [35]:
# Define Search Param
search_params = {'n_components': [5,7,10, 15, 20], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(a)

GridSearchCV(cv=None, error_score=nan,
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='batch',
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=10,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=None,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                                 tota

In [36]:
best_lda_model = model.best_estimator_
print("Best Model's Params: ", model.best_params_)
print("Best Log Likelihood Score: ", model.best_score_)
print("Model Perplexity: ", best_lda_model.perplexity(a))

Best Model's Params:  {'learning_decay': 0.7, 'n_components': 5}
Best Log Likelihood Score:  -3742.6453284297268
Model Perplexity:  22.027044664648376


In [37]:
# Define Search Param
search_params2 = {'n_components': [5,3], 'learning_decay': [.5,.3]}

# Init the Model
lda2 = LatentDirichletAllocation()

# Init Grid Search Class
model2 = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model2.fit(a)

GridSearchCV(cv=None, error_score=nan,
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='batch',
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=10,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=None,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                                 tota

In [38]:
best_lda_model2 = model2.best_estimator_
print("Best Model's Params: ", model2.best_params_)
print("Best Log Likelihood Score: ", model2.best_score_)
print("Model Perplexity: ", best_lda_model2.perplexity(a))

Best Model's Params:  {'learning_decay': 0.9, 'n_components': 5}
Best Log Likelihood Score:  -3747.073577024593
Model Perplexity:  22.033580825213836


In [39]:
n_topics = [3,5,7,10, 15]
log_likelyhoods_5 = [round(gscore.mean_validation_score) for gscore in model.param_grid if gscore.parameters['learning_decay']==0.5]
log_likelyhoods_7 = [round(gscore.mean_validation_score) for gscore in model.param_grid_ if gscore.parameters['learning_decay']==0.7]
log_likelyhoods_9 = [round(gscore.mean_validation_score) for gscore in model.cv_results_ if gscore.parameters['learning_decay']==0.9]

# Show graph
plt.figure(figsize=(12, 8))
plt.plot(n_topics, log_likelyhoods_5, label='0.5')
plt.plot(n_topics, log_likelyhoods_7, label='0.7')
plt.plot(n_topics, log_likelyhoods_9, label='0.9')
plt.title("Choosing Optimal LDA Model")
plt.xlabel("Num Topics")
plt.ylabel("Log Likelyhood Scores")
plt.legend(title='Learning decay', loc='best')
plt.show()

AttributeError: 'str' object has no attribute 'parameters'

In [40]:
import sklearn
>>> sklearn.__version__

'0.22.2.post1'

In [41]:
lda_model = LatentDirichletAllocation(n_components=5,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                         
                                      random_state=100,          # Random state
                                                   # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,
                                      learning_decay=0.5 # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(a)

In [42]:
H=lda_model.components_

In [43]:
W=lda_output

In [44]:
feature_names=count_vec.get_feature_names()

In [45]:
lda_model.perplexity(a)

22.21234814926088

In [46]:
lda_model.score(a)

-16951.244568720947

In [47]:
display_topics(H,W,feature_names,training_data,4,4)

Topic 0:
phonepe aaa i4india india
i4india pmcaresfund phonepe indiafightscorona indiavscorona indiafightscoronavirus phonepe app aaa aa aaa aaa aaa aaa aa aaa aaa aaa aa aaa aa aaa aaa aa aaa aaa aaa aa aaa aaa aaa aaa aa aaa aaa aaa aa
come india let fight coronavirus together join i4india movement donate pmcaresfund phonepe stand together india fight big act small i4india eachrupeecounts donate
asimriaz shehnaazgiil sidnaaz smackdown coronavirus hello india real hero part i4india mvmnt donate pmcaresfund donate phonepe add rs10 u donate even re1 pmo click donate
itaaas time stand india join i4india movement country aa donate pmcaresfund set proudly profile stand together india fight big covid19india pmcaresfund phonepe i4india
Topic 1:
phonepe_ indiafightscorona covid19 narendramodi
do bit contribution pmcares fund fight corona initiative start phonepe_ collect fund minute see donor donate inr phonepe indiafightscorona donatekaronaindia narendramodi
please donate pm releif fund even

In [48]:
raw_data['tweet']

0        Hi Suraj We will not be making any revenue on...
1        PhonePe_ COVID19 Covid_19 Corona Paytm Google...
2        Aap se request hai pls donate ta ke hum bacch...
3        Airtel Payments Bank now offers Bharti AXA Co...
4              phone pe bhi baat hosakti hai common sense
                              ...                        
1166     Here is my small contribution towards PM-CARE...
1167     Paytm vijayshekhar This is not good In a situ...
1168     What Demonetization did to wallets and online...
1169     PhonePe_ A AAA AA BoycottPhonePe CoronavirusP...
1170     CoronaOutbreak MoHFW_INDIA narendramodi myogi...
Name: tweet, Length: 1171, dtype: object

In [50]:
# column names
topicnames = ["Topic" + str(i) for i in range(len(lda_model.components_))]

# index names
docnames= ["Doc" + str(i) for i in range(len(training_data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)


In [51]:
df_document_topic['tweet']=tweets_text

In [53]:
df_document_topic.tweet_text =df_document_topic.tweet_text.dtype(str)

AttributeError: 'DataFrame' object has no attribute 'tweet_text'

In [54]:
type(raw_data['tweet'])

pandas.core.series.Series

In [55]:
df_document_topic.drop('tweet_text',inplace=True,axis=1)

KeyError: "['tweet_text'] not found in axis"

In [56]:
df_document_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic,tweet
Doc0,0.05,0.05,0.05,0.80,0.05,3,Hi Suraj We will not be making any revenue on...
Doc1,0.03,0.38,0.03,0.03,0.54,4,PhonePe_ COVID19 Covid_19 Corona Paytm Google...
Doc2,0.04,0.41,0.04,0.04,0.47,4,Aap se request hai pls donate ta ke hum bacch...
Doc3,0.04,0.04,0.04,0.84,0.04,3,Airtel Payments Bank now offers Bharti AXA Co...
Doc4,0.05,0.05,0.80,0.05,0.05,2,phone pe bhi baat hosakti hai common sense
...,...,...,...,...,...,...,...
Doc1166,0.04,0.84,0.04,0.04,0.04,1,Here is my small contribution towards PM-CARE...
Doc1167,0.04,0.04,0.04,0.04,0.84,4,Paytm vijayshekhar This is not good In a situ...
Doc1168,0.07,0.07,0.07,0.07,0.73,4,What Demonetization did to wallets and online...
Doc1169,0.62,0.22,0.05,0.05,0.05,0,PhonePe_ A AAA AA BoycottPhonePe CoronavirusP...


In [57]:
df_document_topic[df_document_topic.dominant_topic==3]

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic,tweet
Doc0,0.05,0.05,0.05,0.80,0.05,3,Hi Suraj We will not be making any revenue on...
Doc3,0.04,0.04,0.04,0.84,0.04,3,Airtel Payments Bank now offers Bharti AXA Co...
Doc6,0.04,0.04,0.04,0.84,0.04,3,PhonePeSupport PhonePe_ Hi can you provide T ...
Doc7,0.41,0.03,0.03,0.51,0.03,3,In the wake of covid outbreak your donation t...
Doc8,0.04,0.04,0.04,0.84,0.04,3,Random people bulk emailing now for personal ...
...,...,...,...,...,...,...,...
Doc664,0.03,0.03,0.40,0.52,0.03,3,KTRTRS sir let us where to donate for cm reli...
Doc665,0.04,0.04,0.04,0.83,0.04,3,If I contribute times to pm covid fund then p...
Doc668,0.03,0.43,0.03,0.50,0.03,3,My Contribution To Fight Against COVID-19 Ind...
Doc669,0.07,0.26,0.07,0.54,0.07,3,PhonePe_ PhonePeSupport GooglePayIndia Start ...


In [58]:
n=367
print(df_document_topic.iloc[n],df_document_topic['tweet'][n])

Topic0                                                         0.52
Topic1                                                         0.33
Topic2                                                         0.05
Topic3                                                         0.05
Topic4                                                         0.05
dominant_topic                                                    0
tweet              Large brands which have invested heavily on t...
Name: Doc367, dtype: object  Large brands which have invested heavily on the IPL so far include CocaCola Vivo Amazon and PhonePe the franchises alone were on track to notch up AAA500 crore from sponsors collectively ipl2020 covid19 coronavirus


In [59]:
dic={'Topic0':'REQUEST','Topic1':'NOT RELATED','Topic2':'HELP-COVID19','Topic3':'DONATION','Topic4':'INFORMATION'}

In [60]:
df_document_topic.to_csv('lda_result.csv')